In [1]:
import pandas as pd

In [8]:
train_data = pd.read_csv("D:/PGDBA/Competitions/IITG-Gender/train.csv")

In [9]:
train_data.head()

,last_name,first_name,gender,race
0,james,harold d ii,m,black
1,cason,bruce g jr,m,black
2,lane,rommie l,m,black
3,lathan,melvin jr,m,black
4,colon,javier,m,hispanic


In [117]:
type(train_data['first_name'])
train_data.shape[0]
train_data['feature_name'] = train_data['gender']
#train_data['feature_last_vowel']
str(train_data['first_name'].iloc[1]).split()[0]

'bruce'

In [120]:
for i in range(0, train_data.shape[0]):
    train_data['feature_name'].iloc[i]=str(train_data['first_name'].iloc[i]).split()[0]
    if(i%10000 == 0):
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000


In [208]:
train_data['last'] = train_data['feature_name'].str[-1]

In [209]:
def countVowel(col):
    if col in "aeiou":
        return 1
    return 0

train_data['feature_last_vowel'] = train_data['last'].apply(countVowel)

In [122]:
for i in range(0, train_data.shape[0]):
    if(str(train_data['feature_name'].iloc[i]).split()[0][-1] != ''):
        train_data['lastLetter_firstname'][i] = str(train_data['feature_name'].iloc[i]).split()[0][-1]

In [215]:
train_data.dtypes

last_name                object
first_name               object
gender                   object
race                     object
feature_name             object
lastLetter_firstname      int64
last2Letter_firstname     int32
len_firstName             int64
feature_last_vowel        int64
last                     object
dtype: object

In [126]:
train_data['last2Letter_firstname'] = ''
for i in range(0, train_data.shape[0]):
    if(str(train_data['feature_name'].iloc[i]).split()[0][-2:] != ''):
        train_data['last2Letter_firstname'][i] = str(train_data['feature_name'].iloc[i]).split()[0][-2:]

In [129]:
from sklearn.preprocessing import LabelEncoder
labelEnc = LabelEncoder()

In [217]:
train_data['lastLetter_firstname'] = labelEnc.fit_transform(train_data['lastLetter_firstname'])

In [270]:
train_data['last2Letter_firstname'] = labelEnc.fit_transform(train_data['last2Letter_firstname'])
train_data['gender'] = labelEnc.fit_transform(train_data['gender'])
train_data['race'] = labelEnc.fit_transform(train_data['race'])

In [219]:
train_data.tail()

,last_name,first_name,gender,race,feature_name,lastLetter_firstname,last2Letter_firstname,len_firstName,feature_last_vowel,last
85267,chahar,yogesh,m,indian,yogesh,13,292,6,0,h
85268,ostrander,tyler c,m,white,tyler,23,86,5,0,r
85269,lindsey,william b,m,white,william,18,19,7,0,m
85270,pfister,christopher,m,white,christopher,23,86,11,0,r
85271,chugh,sumit,m,indian,sumit,25,145,5,0,t


In [142]:
for i in range (0, len(train_data)):
    train_data['len_firstName'][i] = len(train_data['feature_name'].iloc[i])
    if(i % 10000 == 0):
        print(i)

C:\Users\Bhushan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0
10000
20000
30000
40000
50000
60000
70000
80000


In [279]:
train_data.head()

,last_name,first_name,gender,race,feature_name,lastLetter_firstname,last2Letter_firstname,len_firstName,feature_last_vowel,last
0,james,harold d ii,1,0,harold,9,181,6,0,d
1,cason,bruce g jr,1,0,bruce,10,46,5,1,e
2,lane,rommie l,1,0,rommie,10,130,6,1,e
3,lathan,melvin jr,1,0,melvin,19,139,6,0,n
4,colon,javier,1,1,javier,23,86,6,0,r


In [280]:
from sklearn.model_selection import train_test_split

features_to_use = ['feature_last_vowel', 'lastLetter_firstname', 'last2Letter_firstname', 'len_firstName']

X = train_data[features_to_use]
y = train_data['race']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [281]:
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier

In [282]:
my_model = XGBClassifier(n_estimators=250, learning_rate=0.05)

In [283]:
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.05,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=250, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [286]:
my_predictions = my_model.predict(X_valid)

In [291]:
print(my_predictions)

[3 3 3 ... 3 2 3]


In [294]:
from sklearn.metrics import accuracy_score,f1_score
accuracy = accuracy_score(my_predictions, y_valid)
f1 = f1_score(my_predictions, y_valid, average='weighted')
print(accuracy, f1)

0.5759014951627088 0.620150577418643


In [293]:
y_valid.head()

29255    3
71522    3
678      3
80267    1
82839    3
Name: race, dtype: int32